In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pdb
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import pickle
import torch
import torch.nn.functional as F
import torch.utils.data as data
import pdb
from torch.autograd import Variable
import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from tqdm import tqdm
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics
import pdb
from sklearn import metrics
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics
from torch.autograd import Variable

In [ ]:
def preprocess(series):
        pad = int((series.shape[2] - INPUT_DIM)/2)
        series = series[:,pad:-pad,pad:-pad]
        series = (series-np.min(series))/(np.max(series)-np.min(series))*MAX_PIXEL_VAL
        series = (series - MEAN) / STDDEV
        series = np.stack((series,)*3, axis=1)
        series_float = torch.FloatTensor(series)
        return series_float

In [ ]:
class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.alexnet(pretrained=True)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.squeeze(x, dim=0) # only batch size 1 supported
        x = self.model.features(x)
        x = self.gap(x).view(x.size(0), -1)
        x = torch.max(x, 0, keepdim=True)[0]
        x = self.classifier(x)
        return x

class TripleMRNet(nn.Module):
    def __init__(self, backbone="resnet18", training=True):
        super().__init__()
        self.backbone = backbone
        if self.backbone == "resnet18":
            resnet = models.resnet18(pretrained=training)
            modules = list(resnet.children())[:-1]
            self.axial_net = nn.Sequential(*modules)
            for param in self.axial_net.parameters():
                param.requires_grad = False
        elif self.backbone == "alexnet":
            self.axial_net = models.alexnet(pretrained=training)

        if self.backbone == "resnet18":
            resnet = models.resnet18(pretrained=training)
            modules = list(resnet.children())[:-1]
            self.sagit_net = nn.Sequential(*modules)
            for param in self.sagit_net.parameters():
                param.requires_grad = False
        elif self.backbone == "alexnet":
            self.sagit_net = models.alexnet(pretrained=training)
        
        if self.backbone == "resnet18":
            resnet = models.resnet18(pretrained=training)
            modules = list(resnet.children())[:-1]
            self.coron_net = nn.Sequential(*modules)
            for param in self.coron_net.parameters():
                param.requires_grad = False
        elif self.backbone == "alexnet":
            self.coron_net = models.alexnet(pretrained=training)

        self.gap_axial = nn.AdaptiveAvgPool2d(1)
        self.gap_sagit = nn.AdaptiveAvgPool2d(1)
        self.gap_coron = nn.AdaptiveAvgPool2d(1)
       
        if self.backbone == "resnet18":
            self.classifier = nn.Linear(3*512, 1)
        elif self.backbone == "alexnet":
            self.classifier = nn.Linear(3*256, 1)

    def forward(self, vol_axial, vol_sagit, vol_coron):
        vol_axial = torch.squeeze(vol_axial, dim=0)
        vol_sagit = torch.squeeze(vol_sagit, dim=0)
        vol_coron = torch.squeeze(vol_coron, dim=0)
       
        if self.backbone == "resnet18":
            vol_axial = self.axial_net(vol_axial)
            vol_sagit = self.sagit_net(vol_sagit)
            vol_coron = self.coron_net(vol_coron)
        elif self.backbone == "alexnet":
            vol_axial = self.axial_net.features(vol_axial)
            vol_sagit = self.sagit_net.features(vol_sagit)
            vol_coron = self.coron_net.features(vol_coron)

        vol_axial = self.gap_axial(vol_axial).view(vol_axial.size(0), -1)
        x = torch.max(vol_axial, 0, keepdim=True)[0]
        vol_sagit = self.gap_sagit(vol_sagit).view(vol_sagit.size(0), -1)
        y = torch.max(vol_sagit, 0, keepdim=True)[0]
        vol_coron = self.gap_coron(vol_coron).view(vol_coron.size(0), -1)
        z = torch.max(vol_coron, 0, keepdim=True)[0]

        w = torch.cat((x, y, z), 1)
        out = self.classifier(w)
        return out

In [ ]:
class MRI_alex(nn.Module):
  def __init__(self, training=True):
    super().__init__()
    self.axial_net = models.alexnet(pretrained=training)
    self.sagit_net = models.alexnet(pretrained=training)
    self.coron_net = models.alexnet(pretrained=training)

    self.gap_axial = nn.AdaptiveAvgPool2d(1)
    self.gap_sagit = nn.AdaptiveAvgPool2d(1)
    self.gap_coron = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(3*256, 1)

  def forward(self,vol_axial, vol_sagit, vol_coron):
    vol_axial = torch.squeeze(vol_axial, dim=0)
    vol_sagit = torch.squeeze(vol_sagit, dim=0)
    vol_coron = torch.squeeze(vol_coron, dim=0)

    vol_axial = self.axial_net.features(vol_axial)
    vol_sagit = self.sagit_net.features(vol_sagit)
    vol_coron = self.coron_net.features(vol_coron)

    vol_axial = self.gap_axial(vol_axial).view(vol_axial.size(0), -1)
    x = torch.max(vol_axial, 0, keepdim=True)[0]

    vol_sagit = self.gap_sagit(vol_sagit).view(vol_sagit.size(0), -1)
    y = torch.max(vol_sagit, 0, keepdim=True)[0]

    vol_coron = self.gap_coron(vol_coron).view(vol_coron.size(0), -1)
    z = torch.max(vol_coron, 0, keepdim=True)[0]

    w = torch.cat((x, y, z), 1)
    out = self.classifier(w)
    return out

In [ ]:
INPUT_DIM = 224
MAX_PIXEL_VAL = 255
MEAN = 58.09
STDDEV = 49.73
class Dataset(data.Dataset):
    def __init__(self, datadir, tear_type, use_gpu):
        super().__init__()
        self.use_gpu = use_gpu
        label_dict = {}
        self.paths = []
        abnormal_label_dict = {}
        if datadir[-1]=="/":
            datadir = datadir[:-1]
        self.datadir = datadir
        for i, line in enumerate(open(datadir+'-'+tear_type+'.csv').readlines()):
            line = line.strip().split(',')
            filename = line[0]
            label = line[1]
            label_dict[filename] = int(label)
        

        for i, line in enumerate(open(datadir+'-'+"abnormal"+'.csv').readlines()):
            line = line.strip().split(',')
            filename = line[0]
            label = line[1]
            abnormal_label_dict[filename] = int(label)

        for filename in os.listdir(os.path.join(datadir, "axial")):
            if filename.endswith(".npy"):
                self.paths.append(filename)
        
        self.labels = [label_dict[path.split(".")[0]] for path in self.paths]
        self.abnormal_labels = [abnormal_label_dict[path.split(".")[0]] for path in self.paths]

        if tear_type != "abnormal":
            temp_labels = [self.labels[i] for i in range(len(self.labels)) if self.abnormal_labels[i]==1]
            neg_weight = np.mean(temp_labels)
        else:
            neg_weight = np.mean(self.labels)
        
        self.weights = [neg_weight, 1 - neg_weight]

    def weighted_loss(self, prediction, target):
        weights_npy = np.array([self.weights[int(t[0])] for t in target.data])
        weights_tensor = torch.FloatTensor(weights_npy)
        if self.use_gpu:
            weights_tensor = weights_tensor.cuda()
        loss = F.binary_cross_entropy_with_logits(prediction, target, weight=Variable(weights_tensor))
        return loss

    def __getitem__(self, index):
        filename = self.paths[index]
        vol_axial = np.load(os.path.join(self.datadir, "axial", filename))
        vol_sagit = np.load(os.path.join(self.datadir, "sagittal", filename))
        vol_coron = np.load(os.path.join(self.datadir, "coronal", filename))

        # axial
        vol_axial_tensor = preprocess(vol_axial)
        
        # sagittal
        vol_sagit_tensor = preprocess(vol_sagit)

        # coronal
        vol_coron_tensor = preprocess(vol_coron)

        label_tensor = torch.FloatTensor([self.labels[index]])

        return vol_axial_tensor, vol_sagit_tensor, vol_coron_tensor, label_tensor, self.abnormal_labels[index]

    def __len__(self):
        return len(self.paths)

def load_data(task, use_gpu):
    train_dir = "/content/drive/My Drive/MRNet-v1.0/train"
    valid_dir = "/content/drive/My Drive/MRNet-v1.0/valid"
    
    train_dataset = Dataset(train_dir, task, use_gpu)
    valid_dataset = Dataset(valid_dir, task, use_gpu)

    train_loader = data.DataLoader(train_dataset, batch_size=1, num_workers=11, shuffle=True)
    valid_loader = data.DataLoader(valid_dataset, batch_size=1, num_workers=11, shuffle=False)

    return train_loader, valid_loader

In [ ]:
def run_model(model, loader, train=False, optimizer=None,abnormal_model_path=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        if abnormal_model_path:
            abnormal_model = MRI_alex(train)
            state_dict = torch.load(abnormal_model_path)
            abnormal_model.load_state_dict(state_dict)
            abnormal_model.cuda()
            abnormal_model.eval()
        model.eval()

    total_loss = 0.0
    num_batches = 0

    for batch in tqdm(loader):
        vol_axial, vol_sagit, vol_coron, label, abnormal = batch
        if train:
            if abnormal_model_path and not abnormal:
                continue
            optimizer.zero_grad()
        if loader.dataset.use_gpu:
            vol_axial, vol_sagit, vol_coron = vol_axial.cuda(), vol_sagit.cuda(), vol_coron.cuda()
            label = label.cuda()
        vol_axial, vol_sagit, vol_coron = Variable(vol_axial), Variable(vol_sagit), Variable(vol_coron)
        label = Variable(label)

        logit = model.forward(vol_axial, vol_sagit, vol_coron)

        loss = loader.dataset.weighted_loss(logit, label)
        total_loss += loss.item()

        pred = torch.sigmoid(logit)

        pred_npy = pred.data.cpu().numpy()[0][0]

        if abnormal_model_path and not train:
            abnormal_logit = abnormal_model.forward(vol_axial,vol_sagit,vol_coron)
            abnormal_pred = torch.sigmoid(abnormal_logit)
            abnormal_pred_npy = abnormal_pred.data.cpu().numpy()[0][0]
            pred_npy = pred_npy * abnormal_pred_npy

        label_npy = label.data.cpu().numpy()[0][0]

        preds.append(pred_npy)
        labels.append(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches
    
    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    if abnormal_model_path and not train:
        del abnormal_model

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, diagnosis, use_gpu):
    train_loader, valid_loader, test_loader = load_data(diagnosis, use_gpu)

    model = MRI_alex()
    state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
    model.load_state_dict(state_dict)

    if use_gpu:
        model = model.cuda()

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")

    loss, auc, preds, labels = run_model(model, loader)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

In [ ]:
def train(rundir, task, epochs, learning_rate, use_gpu,model,abnormal_model_path=None):
    train_loader, valid_loader = load_data(task, use_gpu)
    
    model = model
    '''for dirpath, dirnames, files in os.walk(rundir):
        if not files:
            break
        max_epoch = 0
        model_path = None
        for fname in files:
            if fname.endswith(".json"):
                continue
            ep = int(fname[27:])
            if ep >= max_epoch:
                max_epoch = ep
                model_path = os.path.join(dirpath, fname)
        
        if model_path:
            state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
            model.load_state_dict(state_dict)
'''
    if use_gpu:
        model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=1e-4)

    best_val_loss = float('inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer,abnormal_model_path=abnormal_model_path)

        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader,abnormal_model_path=abnormal_model_path)
        
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')

        scheduler.step(val_loss)

In [ ]:
model_acl=MRI_alex(training=True)
model_men=MRI_alex(training=True)
model_ab=MRI_alex(training=True)

In [ ]:
train('/content/drive/My Drive/MRNet-v1.0/Untitled Folder',task='abnormal',epochs=20,learning_rate=1e-5,use_gpu='cuda',model=model_ab)

  0%|          | 0/1130 [00:00<?, ?it/s]

starting epoch 1. time passed: 0:00:00.000014


  0%|          | 0/120 [00:00<?, ?it/s]

train loss: 0.1773
train AUC: 0.7710


  0%|          | 0/1130 [00:00<?, ?it/s]

...

valid loss: 0.1413
valid AUC: 0.9448
starting epoch 20. time passed: 1:20:03.547185


  0%|          | 0/120 [00:00<?, ?it/s]

train loss: 0.0714
train AUC: 0.9722


100%|██████████| 120/120 [00:28<00:00,  4.21it/s]

valid loss: 0.1737
valid AUC: 0.9465


In [ ]:
torch.save(model_ab.state_dict(), '/content/drive/My Drive/MRNet-v1.0/models/abnormal2.pt')

In [ ]:
model_ab

In [ ]:
def preprocess(series):
        pad = int((series.shape[2] - INPUT_DIM)/2)
        series = series[:,pad:-pad,pad:-pad]
        series = (series-np.min(series))/(np.max(series)-np.min(series))*MAX_PIXEL_VAL
        series = (series - MEAN) / STDDEV
        series = np.stack((series,)*3, axis=1)
        series_float = torch.FloatTensor(series)
        return series_float

def get_study(axial_path, sagit_path, coron_path):
    vol_axial = np.load(axial_path)
    vol_sagit = np.load(sagit_path)
    vol_coron = np.load(coron_path)
    # axial
    vol_axial_tensor = preprocess(vol_axial)
    # sagittal
    vol_sagit_tensor =preprocess(vol_sagit)
    # coronal
    vol_coron_tensor = preprocess(vol_coron)
    
    return {"axial": vol_axial_tensor,
            "sagit": vol_sagit_tensor,
            "coron": vol_coron_tensor}


def get_prediction(model, tensors, abnormality_prior=None):
    vol_axial = tensors["axial"].cuda()
    vol_sagit = tensors["sagit"].cuda()
    vol_coron = tensors["coron"].cuda()

    vol_axial = Variable(vol_axial)
    vol_sagit = Variable(vol_sagit)
    vol_coron = Variable(vol_coron)

    logit = model.forward(vol_axial, vol_sagit, vol_coron)
    pred = torch.sigmoid(logit)
    pred_npy = pred.data.cpu().numpy()[0][0]
    
    if abnormality_prior:
        pred_npy = pred_npy * abnormality_prior

    return pred_npy


def lets_predict3()
    # Assuming that the input csv has all three views for each ID.
    # And that entries are sorted by ID.
    views = []
    for i, fpath in enumerate(open(input_csv_path).readlines()):
        if "axial" in fpath:
            axial_path = fpath.strip()
        elif "sagittal" in fpath:
            sagit_path = fpath.strip()
        elif "coronal" in fpath:
            coron_path = fpath.strip()
        if i%3==2:
            views.append(get_study(axial_path, sagit_path, coron_path))


    # Loading all models
    abnormal_model_path = "src/abnormal_triple_alex/val0.1071_train0.0868_epoch8"
    acl_model_path = "src/acl_triple_alex/val0.1310_train0.0504_epoch30"
    meniscal_model_path = "src/meniscal_triple_alex/val0.2645_train0.1142_epoch22"


    # Getting predictions
    abnormality = []
    acl_tear = []
    meniscal_tear = []
    
    abnormal_model = MRI_alex( training=False)
    state_dict = torch.load(abnormal_model_path)
    abnormal_model.load_state_dict(state_dict)
    abnormal_model.cuda()
    abnormal_model.eval()
    for study in views:
        abnormality.append(get_prediction(
                abnormal_model,
                study,
                abnormality_prior=None))
    del abnormal_model

    acl_model = MRI_alex( training=False)
    state_dict = torch.load(acl_model_path)
    acl_model.load_state_dict(state_dict)
    acl_model.cuda()
    acl_model.eval()
    for idx,study in enumerate(views):
        acl_tear.append(get_prediction(
                acl_model,
                study,
                abnormality_prior=abnormality[idx]))
    del acl_model

    meniscal_model = MRI_alex(training=False)
    state_dict = torch.load(meniscal_model_path)
    meniscal_model.load_state_dict(state_dict)
    meniscal_model.cuda()
    meniscal_model.eval()
    for idx,study in enumerate(views):
        meniscal_tear.append(get_prediction(
                meniscal_model,
                study,
                abnormality_prior=abnormality[idx]))
    del meniscal_model
    

    with open(preds_csv_path, "w") as csv_file:        
        for i in range(len(abnormality)):
            csv_file.write(",".join(
                [str(abnormality[i]), str(acl_tear[i]), str(meniscal_tear[i])]))
            csv_file.write("\n"

In [ ]:
get_prediction(model_ab,get_study("/content/drive/My Drive/MRNet-v1.0/valid/axial/1135.npy","/content/drive/My Drive/MRNet-v1.0/valid/sagittal/1135.npy","/content/drive/My Drive/MRNet-v1.0/valid/coronal/1135.npy"))

0.6930317